In [50]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from numpy import linalg as la
from tabulate import tabulate
import linearpaneldata as lpd   # simple routines to do linear FE and Pooled OLS regressions
import indexmodels as im
from indexmodels import *       # objective functions etc. for estimation of panel data binary response models
import mestim as M       


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
df = pd.read_csv('eco_data.csv', delimiter=',')
Nobs=df['Portfolio_type'].count()
df['const']=np.ones((Nobs,1))
N = df.shape
print(N)

(20, 8)


In [84]:
df

rolling_window  Portfolio_type  Expected Return   ESG Score     E Score   
0                1               1           0.3400  119.430000  119.980000  \
1                1               2          -0.1028   86.588182   73.936568   
2                2               1           0.3400   41.548944   34.131990   
3                2               2          -0.1646   80.037791   66.579101   
4                3               1           0.3400   37.544536   29.613484   
5                3               2          -0.0938   73.784234   61.374848   
6                4               1           0.2434   39.509550   27.587139   
7                4               2           0.2104   41.786324   29.549139   
8                5               1           0.1696   43.545538   31.149471   
9                5               2           0.1450   45.633473   32.991023   
10               6               1           0.2230   40.565975   27.943627   
11               6               2           0.1684   45.735215   32.329116   
12               7               1           0.3292   40.470203   19.892804   
13               7               2           0.2608   43.836021   24.853601   
14               8               1           0.3400   42.480720   19.461360   
15               8               2           0.1444   49.536635   30.415093   
16               9               1           0.2626   46.517368   23.240638   
17               9               2           0.1948   47.806761   26.684597   
18              10               1           0.3400   50.932617   38.056315   
19              10               2           0.1042   46.902168   32.933534   

       S Score     G Score  const  
0   113.760000  124.515615    1.0  
1    86.124522   99.659654    1.0  
2    -0.784786   91.096681    1.0  
3    78.363849   95.118843    1.0  
4    10.567435   72.312745    1.0  
5    68.456390   91.457956    1.0  
6     9.052245   81.721445    1.0  
7    13.153046   82.495613    1.0  
8    14.733164   84.591526    1.0  
9    19.144886   84.611036    1.0  
10    6.868322   86.702185    1.0  
11   17.374001   87.338633    1.0  
12  -10.448195  111.683307    1.0  
13    0.892049  105.518606    1.0  
14   -4.137320  111.845464    1.0  
15   22.223322   95.793093    1.0  
16   11.456885  104.629779    1.0  
17   18.470493   98.071646    1.0  
18   19.841859   94.726039    1.0  
19   29.736294   77.918098    1.0

In [53]:
print(df['S Score'])

0     113.760000
1      -0.784786
2      10.567435
3       9.052245
4      14.733164
5       6.868322
6     -10.448195
7      -4.137320
8      11.456885
9      19.841859
10     86.124522
11     78.363849
12     68.456390
13     13.153046
14     19.144886
15     17.374001
16      0.892049
17     22.223322
18     18.470493
19     29.736294
Name: S Score, dtype: float64


In [70]:
print("The variance of \"Expected Return\"=",((df.groupby(["rolling_window"])["Expected Return"].var()).mean()))

print("The variance of \"ESG Score\"=",((df.groupby(["rolling_window"])["ESG Score"].var()).mean()))

print("The variance of \"E Score\"=",((df.groupby(["rolling_window"])["E Score"].var()).mean()))

print("The variance of \"S Score\"=",((df.groupby(["rolling_window"])["S Score"].var()).mean()))

print("The variance of \"G Score\"=",((df.groupby(["rolling_window"])["G Score"].var()).mean()))


The variance of "Expected Return"= 0.037334358
The variance of "ESG Score"= 199.42889915608401
The variance of "E Score"= 219.53838563764288
The variance of "S Score"= 574.8290718107381
The variance of "G Score"= 81.13693366934072


In [55]:
y_it=df['Expected Return']                           # binary response variable: Labor force participation
x_it=df['ESG Score']                 # time varying explanatory varibles  
x_t=['E Score']   # time dummies (leave one out)
x_i=['S Score']   # time constant explanatory varibles (not used in FE regressions)
x_k = ['G Score']
groupvar = 'Portfolio_type'

In [67]:
lpm_pols=lpd.estim(df, y_it, xvar=x_it+x_t+x_i, groupvar='Portfolio_type', method='pols', cov_type='robust')
lpm_fe=lpd.estim(df, y_it, xvar=x_it+x_t , groupvar='Portfolio_type', method='fe', cov_type='robust')



Specification: Pooled OLS Panel Regression
Dep. var. : ['ESG Score'] 

parnames         b_hat          se    t-values
----------  ----------  ----------  ----------
E Score         0.3331      0.0000  18151.3721
S Score         0.3327      0.0000  26454.0946
G Score         0.3342      0.0000  62081.2831
# of groups:       10
# of observations: 20 


Specification: Linear Fixed Effects Regression
Dep. var. : ['ESG Score'] 

parnames         b_hat          se    t-values
----------  ----------  ----------  ----------
E Score         0.6116      0.0403     15.1679
S Score         0.2272      0.0302      7.5145
# of groups:       10
# of observations: 20 



In [64]:
pooled_probit = im.pooled(df, y_it, xvar=x_it+x_t+x_i + ['const'] , groupvar='Portfolio_type', model='probit')


Pooled probit
Dep. var. : ['Expected Return'] 

parnames      theta_hat          se    t-values         jac         APE
----------  -----------  ----------  ----------  ----------  ----------
ESG Score       0.00511     0.06105     0.08376    -0.00002     0.00100
E Score         0.09795     0.11883     0.82426    -0.00003     0.01912
S Score        -0.09553     0.08768    -1.08949    -0.00001    -0.01865
const          -2.91540     1.69896    -1.71599     0.00000    -0.56919

# of groups:      : 10
# of observations : 20
# log-likelihood. : -7.932382525952358 

Iteration info: 32 iterations, 33 evaluations of objective, and 33 evaluations of gradients
Elapsed time: 0.0435 seconds



In [63]:
res_rep=rand_effect(df, y_it, xvar = x_it+x_i +x_t+ ['const'], groupvar = 'Portfolio_type', model='probit',cov_type='sandwich', deriv=2)

Random effects probit
Dep. var. : ['Expected Return'] 

parnames      theta_hat          se    t-values         jac         APE
----------  -----------  ----------  ----------  ----------  ----------
ESG Score      -0.00710     0.50314    -0.01411    -2.24894    -0.00172
S Score        -0.00505     0.14202    -0.03557     0.76852    -0.00122
E Score        -0.00547     0.29944    -0.01828    -3.02984    -0.00133
const          -0.00012    12.34965    -0.00001    -0.01920    -0.00003
sigma_a         0.99988     0.71477     1.39888     0.63176     0.24215

# of groups:      : 10
# of observations : 20
# log-likelihood. : -8.972262869966373 

Iteration info: 29 iterations, 30 evaluations of objective, and 2 evaluations of gradients
Elapsed time: 0.0605 seconds



In [97]:
from linearmodels.datasets import jobtraining
data_tmp = jobtraining.load()
year_tmp = pd.Categorical(data_tmp.year)
data_tmp = data_tmp.set_index(['fcode', 'year'])
data_tmp['year'] = year_tmp

data_tmp

employ       sales   avgsal  scrap  rework  tothrs  union  grant   
fcode  year                                                                     
410032 1987   100.0  47000000.0  35000.0    NaN     NaN    12.0      0      0  \
       1988   131.0  43000000.0  37000.0    NaN     NaN     8.0      0      0   
       1989   123.0  49000000.0  39000.0    NaN     NaN     8.0      0      0   
410440 1987    12.0   1560000.0  10500.0    NaN     NaN    12.0      0      0   
       1988    13.0   1970000.0  11000.0    NaN     NaN    12.0      0      0   
...             ...         ...      ...    ...     ...     ...    ...    ...   
419483 1988   108.0  11500000.0  14810.0   25.0     NaN     0.0      1      0   
       1989   129.0  12000000.0  14227.0   30.0     NaN    20.0      1      0   
419486 1987    80.0   7000000.0  16000.0    NaN     NaN     0.0      0      0   
       1988    90.0   8500000.0  17000.0    NaN     NaN     0.0      0      0   
       1989   100.0   9900000.0  18000.0    NaN     NaN    40.0      0      1   

             d89  d88  ...   clscrap  cgrant  clemploy   clsales    lavgsal   
fcode  year            ...                                                    
410032 1987    0    0  ...       NaN       0       NaN       NaN  10.463100  \
       1988    0    1  ...       NaN       0  0.270027 -0.088949  10.518670   
       1989    1    0  ...       NaN       0 -0.063013  0.130621  10.571320   
410440 1987    0    0  ...       NaN       0       NaN       NaN   9.259130   
       1988    0    1  ...       NaN       0  0.080043  0.233347   9.305651   
...          ...  ...  ...       ...     ...       ...       ...        ...   
419483 1988    0    1  ...  0.223144       0 -0.208218  0.044453   9.603058   
       1989    1    0  ...  0.182321       0  0.177681  0.042559   9.562897   
419486 1987    0    0  ...       NaN       0       NaN       NaN   9.680344   
       1988    0    1  ...       NaN       0  0.117783  0.194157   9.740969   
       1989    1    0  ...       NaN       1  0.105360  0.152469   9.798127   

             clavgsal  cgrant_1    chrsemp  clhrsemp  year  
fcode  year                                                 
410032 1987       NaN       NaN        NaN       NaN  1987  
       1988  0.055570       0.0  -8.946565 -1.165385  1988  
       1989  0.052644       0.0   0.198597  0.047832  1989  
410440 1987       NaN       NaN        NaN       NaN  1987  
       1988  0.046520       0.0   0.000000  0.000000  1988  
...               ...       ...        ...       ...   ...  
419483 1988  0.059321       0.0   0.000000  0.000000  1988  
       1989 -0.040161       0.0   3.100775  1.411176  1989  
419486 1987       NaN       NaN        NaN       NaN  1987  
       1988  0.060625       0.0   0.000000  0.000000  1988  
       1989  0.057159       0.0  36.000000  3.610918  1989  

[471 rows x 29 columns]

In [57]:
import numpy as np
import pandas as pd
from linearmodels import PanelOLS
from linearmodels import RandomEffects

In [86]:
from linearmodels.datasets import jobtraining
year = pd.Categorical(df.Portfolio_type)
data = df.set_index(['rolling_window', 'Portfolio_type'])
data['year'] = year

In [96]:
data

Expected Return   ESG Score     E Score   
rolling_window Portfolio_type                                            
1              1                        0.3400  119.430000  119.980000  \
               2                       -0.1028   86.588182   73.936568   
2              1                        0.3400   41.548944   34.131990   
               2                       -0.1646   80.037791   66.579101   
3              1                        0.3400   37.544536   29.613484   
               2                       -0.0938   73.784234   61.374848   
4              1                        0.2434   39.509550   27.587139   
               2                        0.2104   41.786324   29.549139   
5              1                        0.1696   43.545538   31.149471   
               2                        0.1450   45.633473   32.991023   
6              1                        0.2230   40.565975   27.943627   
               2                        0.1684   45.735215   32.329116   
7              1                        0.3292   40.470203   19.892804   
               2                        0.2608   43.836021   24.853601   
8              1                        0.3400   42.480720   19.461360   
               2                        0.1444   49.536635   30.415093   
9              1                        0.2626   46.517368   23.240638   
               2                        0.1948   47.806761   26.684597   
10             1                        0.3400   50.932617   38.056315   
               2                        0.1042   46.902168   32.933534   

                                  S Score     G Score  const year  
rolling_window Portfolio_type                                      
1              1               113.760000  124.515615    1.0    1  
               2                86.124522   99.659654    1.0    2  
2              1                -0.784786   91.096681    1.0    1  
               2                78.363849   95.118843    1.0    2  
3              1                10.567435   72.312745    1.0    1  
               2                68.456390   91.457956    1.0    2  
4              1                 9.052245   81.721445    1.0    1  
               2                13.153046   82.495613    1.0    2  
5              1                14.733164   84.591526    1.0    1  
               2                19.144886   84.611036    1.0    2  
6              1                 6.868322   86.702185    1.0    1  
               2                17.374001   87.338633    1.0    2  
7              1               -10.448195  111.683307    1.0    1  
               2                 0.892049  105.518606    1.0    2  
8              1                -4.137320  111.845464    1.0    1  
               2                22.223322   95.793093    1.0    2  
9              1                11.456885  104.629779    1.0    1  
               2                18.470493   98.071646    1.0    2  
10             1                19.841859   94.726039    1.0    1  
               2                29.736294   77.918098    1.0    2

In [90]:
import statsmodels.api as sm

In [94]:
data

Expected Return   ESG Score     E Score   
rolling_window Portfolio_type                                            
1              1                        0.3400  119.430000  119.980000  \
               2                       -0.1028   86.588182   73.936568   
2              1                        0.3400   41.548944   34.131990   
               2                       -0.1646   80.037791   66.579101   
3              1                        0.3400   37.544536   29.613484   
               2                       -0.0938   73.784234   61.374848   
4              1                        0.2434   39.509550   27.587139   
               2                        0.2104   41.786324   29.549139   
5              1                        0.1696   43.545538   31.149471   
               2                        0.1450   45.633473   32.991023   
6              1                        0.2230   40.565975   27.943627   
               2                        0.1684   45.735215   32.329116   
7              1                        0.3292   40.470203   19.892804   
               2                        0.2608   43.836021   24.853601   
8              1                        0.3400   42.480720   19.461360   
               2                        0.1444   49.536635   30.415093   
9              1                        0.2626   46.517368   23.240638   
               2                        0.1948   47.806761   26.684597   
10             1                        0.3400   50.932617   38.056315   
               2                        0.1042   46.902168   32.933534   

                                  S Score     G Score  const year  
rolling_window Portfolio_type                                      
1              1               113.760000  124.515615    1.0    1  
               2                86.124522   99.659654    1.0    2  
2              1                -0.784786   91.096681    1.0    1  
               2                78.363849   95.118843    1.0    2  
3              1                10.567435   72.312745    1.0    1  
               2                68.456390   91.457956    1.0    2  
4              1                 9.052245   81.721445    1.0    1  
               2                13.153046   82.495613    1.0    2  
5              1                14.733164   84.591526    1.0    1  
               2                19.144886   84.611036    1.0    2  
6              1                 6.868322   86.702185    1.0    1  
               2                17.374001   87.338633    1.0    2  
7              1               -10.448195  111.683307    1.0    1  
               2                 0.892049  105.518606    1.0    2  
8              1                -4.137320  111.845464    1.0    1  
               2                22.223322   95.793093    1.0    2  
9              1                11.456885  104.629779    1.0    1  
               2                18.470493   98.071646    1.0    2  
10             1                19.841859   94.726039    1.0    1  
               2                29.736294   77.918098    1.0    2

In [92]:
exog_vars = ['E Score', 'G Score']
exog = sm.add_constant(data[exog_vars])
mod = RandomEffects(data, exog)
re_res = mod.fit()
print(re_res)

ValueError: Length mismatch: Expected axis has 7 elements, new values have 1 elements